<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/Final_Large%20Latent%20Space%20-%20VAE_Train_and_Save_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
#from keras.utils import plot_model
#import matplotlib.pyplot as plt

#window1 = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-500.npy')
#window2 = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-1000.npy')
#window = np.concatenate((window1, window2), axis=0)
#train_data = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TRAIN.npy')
#test_data = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TEST.npy')



#get data
train_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TRAIN_hourly.npy')
test_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TEST_hourly.npy')
#window_label = np.load(r'/content/drive/MyDrive/PHD/2021/DATAFILES/WINDOW_LABELLED.npy')
n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]



#create sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

#Build the encoder
latent_dim = 20
intermediate_dim = 1000
encoder_inputs =  layers.Input(shape=(window_size, n_features),name="encoder_input")
x = layers.LSTM(intermediate_dim, activation='tanh', name="lstm1" )(encoder_inputs)
x1 = layers.Dense(intermediate_dim, name="dense" )(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x1)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x1)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


#Dcoder

inp_z = Input(shape=(latent_dim,),name="decoder")
x1 = layers.RepeatVector(window_size, name="repeatvect")(inp_z)
x2= layers.Dense(intermediate_dim,  name="Dense2")(x1)
x3= layers.LSTM(intermediate_dim,activation='tanh', return_sequences=True, name="lstm2")(x2)
decode_out = layers.TimeDistributed(Dense(n_features), name="decodeout")(x3)
#decode_out = layers.LSTM(n_features,name='decodeout', return_sequences=True)(x2) #Alternative
decoder = keras.Model(inp_z, decode_out, name="decoder")
decoder.summary()
#Parameters
n_epochs = 500 # total number of epochs
klstart = 10 # The number of epochs at which KL loss should be included
kl_annealtime = n_epochs-klstart
# the starting value of weight is 0
# define it as a keras backend variable
weight = K.variable(0.0)

#Define the VAE as a Model with a custom train_step

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,

        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction), axis=-1),keepdims=True
                )

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            K.print_tensor(weight)
            total_loss = reconstruction_loss + (weight*kl_loss)
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):

            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction), axis=-1),keepdims=True
                )

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

            total_loss = reconstruction_loss + kl_loss

            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
                  }



# CALLBACKS
es = keras.callbacks.EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='loss', mode='auto', restore_best_weights=True)

class AnnealingCallback(Callback):
    def __init__(self, weight):
        self.weight = weight
    def on_epoch_end (self, epoch, logs={}):
        if epoch > klstart :
            new_weight = min(K.get_value(self.weight) + (1./ kl_annealtime), 1.)
            K.set_value(self.weight, new_weight)
        print ("Current KL Weight is " + str(K.get_value(self.weight)))

#Train the VAE

vae = VAE(encoder, decoder)

vae.compile(optimizer=keras.optimizers.Adam(clipnorm=1))
history=vae.fit(train_data,
                 epochs=n_epochs,
                 batch_size=50,
                 validation_split=0.3,
                 callbacks=[es,AnnealingCallback(weight)])


encoder.save(r'/content/drive/MyDrive/PHD/2021/vae-encoder-saved-round3.model')
decoder.save(r'/content/drive/MyDrive/PHD/2021/vae-decoder-saved-round3.model')

encoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2021/vae-encoder-saved-round3.model')
decoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2021/vae-decoder-saved-round3.model')

e =
#vae2.evaluate(test_data)

# Plot training & validation loss values
plt.plot(history.history['loss'], label='Training loss')
#plt.plot(history.history['val_loss'], label='validation loss')
#plt.plot(history.history['kl_loss'], label='kl_Loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['loss','val_loss'], loc='upper left')
plt.show()

plt.plot(history.history['kl_loss'], label='KL loss')
#plt.plot(history.history['total_val_loss'], label='reconstruction_loss')
#plt.plot(history.history['kl_loss'], label='kl_Loss')
plt.title('Model loss')
plt.ylabel('KL Loss')
plt.xlabel('Epoch')
plt.legend(['kl_loss'], loc='upper left')
plt.show()

#PLOT TRAIN RECONSTRUCTION
X_test_encoded = encoder.predict(train_data[0:100,:,:])
X_test_predict = decoder(X_test_encoded[2])
plt.suptitle('Sub sequence plotting', fontsize='30')
plt.xlabel('Time', fontsize ='20')
plt.ylabel('Feature 3', fontsize='20')
plt.plot(train_data[50,:,2],"r")
plt.plot(X_test_predict[50,:,2],"b")
plt.show()

#PLOT TEST RECONSTRUCTION
X_test_encoded = encoder.predict(test_data[0:10,:,:])
X_test_predict = decoder(X_test_encoded[2])
plt.suptitle('Sub sequence plotting', fontsize='30')
plt.xlabel('Time', fontsize ='20')
plt.ylabel('Feature 1', fontsize='20')
plt.plot(test_data[0,:,2],"r")
plt.plot(X_test_predict[0,:,2],"b")
plt.show()

fig = px.scatter(None, x=X_test_encoded[2][:,0], y=X_test_encoded[2][:,1],opacity=1, color=window_label.astype(str))
fig.update_layout(dict(plot_bgcolor = 'white'))
fig.update_traces(marker=dict(size=2))
fig.show()

   #vae2.evaluate(test_data)

# serialize encoder model to JSON
#model_json_encoder = encoder.to_json()
#with open("model_encoder.json", "w") as json_file:
#    json_file.write(model_json_encoder)
# serialize encoder weights to HDF5
#encoder.save_weights("model_encoder.h5")

# serialize whole model to JSON
#vae.save(r'/content/drive/MyDrive/PHD/2021/vae-saved.model')
#modelVAEjson = vae.to_json()
#with open("modelVAE.json", "w") as json_file:
# json_file.write(modelVAE_json)
# serialize weights to HDF5
#vae.save_weights("modelVAE.h5")
#print("Saved model to disk")

Streaming output truncated to the last 5000 lines.
257/435 [================>.............] - ETA: 2s - loss: 11.0936 - reconstruction_loss: 10.2359 - kl_loss: 4.0972 0.202222377
 0.202222377
 0.202222377
 0.202222377
261/435 [=================>............] - ETA: 2s - loss: 11.0932 - reconstruction_loss: 10.2355 - kl_loss: 4.0997 0.202222377
 0.202222377
 0.202222377
 0.202222377
265/435 [=================>............] - ETA: 2s - loss: 11.0927 - reconstruction_loss: 10.2306 - kl_loss: 4.1009 0.202222377
 0.202222377
 0.202222377
 0.202222377
269/435 [=================>............] - ETA: 2s - loss: 11.0923 - reconstruction_loss: 10.2371 - kl_loss: 4.1027 0.202222377
 0.202222377
 0.202222377
 0.202222377
273/435 [=================>............] - ETA: 2s - loss: 11.0919 - reconstruction_loss: 10.2374 - kl_loss: 4.1035 0.202222377
 0.202222377
 0.202222377
 0.202222377
277/435 [==================>...........] - ETA: 2s - loss: 11.0916 - reconstruction_loss: 10.2366 - kl_loss: 4.102